In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [3]:
import requests

pl_season = 2022

url = f"https://fbref.com/en/comps/9/{pl_season}-{pl_season + 1}/stats/{pl_season}-{pl_season + 1}-Premier-League-Stats"
response = requests.get(url)
response
url

'https://fbref.com/en/comps/9/2022-2023/stats/2022-2023-Premier-League-Stats'

In [4]:
bs1 = BeautifulSoup(response.content, 'html.parser')
text_of_bs1 = str(bs1)
usable_bs2 = text_of_bs1.replace("<!--"," ").replace("-->", " ")
soup = BeautifulSoup(usable_bs2, 'html.parser')

In [5]:
url_list = soup.find_all('a')
url_list = [url for url in url_list if str(url).find('player') > 0]
url_list = [url for url in url_list if str(url).find('matchlogs') < 0]
url_list = [str(url) for url in url_list]
url_list

['<a href="/en/players/">Players</a>',
 '<a href="/en/players/1f44ac21/Erling-Haaland">Erling Haaland</a>',
 '<a href="/en/players/e46012d4/Kevin-De-Bruyne">Kevin De Bruyne</a>',
 '<a href="/en/players/7ba6d84e/David-de-Gea">David de Gea</a>',
 '<a href="/en/players/5bc43860/Brenden-Aaronson">Brenden Aaronson</a>',
 '<a href="/en/players/56628958/George-Abbott">George Abbott</a>',
 '<a href="/en/players/f2bf1b0f/Che-Adams">Che Adams</a>',
 '<a href="/en/players/2b09d998/Tyler-Adams">Tyler Adams</a>',
 '<a href="/en/players/c81d773d/Tosin-Adarabioyo">Tosin Adarabioyo</a>',
 '<a href="/en/players/288e1e13/Nayef-Aguerd">Nayef Aguerd</a>',
 '<a href="/en/players/a6c58494/Naouirou-Ahamada">Naouirou Ahamada</a>',
 '<a href="/en/players/9b398aea/Rayan-Ait-Nouri">Rayan Aït Nouri</a>',
 '<a href="/en/players/a8c0acb7/Kristoffer-Ajer">Kristoffer Ajer</a>',
 '<a href="/en/players/89ac64a6/Manuel-Akanji">Manuel Akanji</a>',
 '<a href="/en/players/eaeca114/Nathan-Ake">Nathan Aké</a>',
 '<a href="/e

In [6]:
url_id_list = []
tag_list = []
excluded_indexes = []
for i in range(0, len(url_list)):
    if len(url_list[i]) > 40 and len(url_list[i]) < 100:
        url_id_list.append(url_list[i].split('"')[1].split('/')[3])
        tag_list.append(url_list[i].split('"')[1].split('/')[4])
    else:
        excluded_indexes.append(i)

In [7]:
tag_list = tag_list[0:(excluded_indexes[1]-1)]
url_id_list = url_id_list[0:(excluded_indexes[1]-1)]

In [8]:
url_id_values = pd.Series(url_id_list)
tag_values = pd.Series(tag_list)
df = pd.DataFrame()
df['url_id'] = url_id_values
df['tag'] = tag_values
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

,url_id,tag
0,1f44ac21,Erling-Haaland
1,e46012d4,Kevin-De-Bruyne
2,7ba6d84e,David-de-Gea
3,5bc43860,Brenden-Aaronson
4,56628958,George-Abbott
...,...,...
549,e8271ad9,Jordan-Zemura
550,51cf8561,Oleksandr-Zinchenko
551,6622454d,Hakim-Ziyech
552,ce4246f5,Kurt-Zouma


In [9]:
fbref_list = []
for x in range(0,len(df)):
    url_name = f"https://fbref.com/en/players/{df.iloc[x]['url_id']}/{df.iloc[x]['tag']}"
    reply = requests.get(url_name)
    soup2 = BeautifulSoup(reply.content, 'html.parser')
    try:
        name1 = soup2.select('div:nth-child(2) > h1')[0].text.split('\n')[1]
    except:
        name1 = None
    try:
        name2 = soup2.select('div:nth-child(2) > p:nth-child(2) > strong')[0].text
    except:
        name2 = None
    if name2 == 'Position:':
        name2 = name1
    fbref_list.append(name2)
    sleep(3)

KeyboardInterrupt: 

In [ ]:
len(fbref_list)

554

In [ ]:
fbref_values = pd.Series(fbref_list)
df['fbref_name'] = fbref_values

In [ ]:
# url_name = "https://fbref.com/en/players/9228b07c/Tom-Davies"
# reply = requests.get(url_name)
# soup2 = BeautifulSoup(reply.content, 'html.parser')
# name1 = soup2.select('div:nth-child(2) > h1')[0].text.split('\n')[1]
# name2 = soup2.select('div:nth-child(2) > p:nth-child(2) > strong')[0].text
# if name2 == 'Position:':
#         name2 = name1

In [ ]:
# for x in range(0,len(duplicated_df)):
#     url_dup = f"https://fbref.com/en/players/{duplicated_df.iloc[x]['url_id']}/matchlogs/2021-2022/c9/summary/{duplicated_df.iloc[x]['tag']}-Match-Log"
#     reply = requests.get(url_dup)
#     soup2 = BeautifulSoup(reply.content, 'html.parser')
#     name = soup2.select('div:nth-child(2) > p:nth-child(2) > strong')[0].text
#     name_list.append(name)

In [ ]:
# (f"https://fbref.com/en/players/{df.iloc[x]['url_id']}/matchlogs/2020-2021/c9/summary/{df.iloc[x]['tag']}-Match-Log")

In [ ]:
player_id_df = pd.read_csv(f'../data/{pl_season}-{pl_season - 1999}/player_idlist.csv')

In [ ]:
player_id_df['full_name'] = player_id_df['first_name'] + ' ' + player_id_df['second_name']

In [ ]:
df['key'] = 0
player_id_df['key'] = 0
merge_df = pd.merge(df, player_id_df, on='key')

In [ ]:
merge_df = merge_df.dropna()

In [ ]:
merge_df

,url_id,tag,fbref_name,key,first_name,second_name,id,full_name
0,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Granit,Xhaka,3,Granit Xhaka
1,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Mohamed,Elneny,4,Mohamed Elneny
2,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Rob,Holding,5,Rob Holding
3,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Thomas,Partey,6,Thomas Partey
4,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Martin,Ødegaard,7,Martin Ødegaard
...,...,...,...,...,...,...,...,...
431007,79300479,Martin-Odegaard,Martin Ødegaard,0,Matheus,Santos Carneiro Da Cunha,682,Matheus Santos Carneiro Da Cunha
431008,79300479,Martin-Odegaard,Martin Ødegaard,0,Mario,Lemina,698,Mario Lemina
431009,79300479,Martin-Odegaard,Martin Ødegaard,0,Pablo,Sarabia,704,Pablo Sarabia
431010,79300479,Martin-Odegaard,Martin Ødegaard,0,Daniel,Bentley,717,Daniel Bentley


In [ ]:
def filter_name(row):
    split1 = row.fbref_name.split()
    split2 = row.full_name.split()
    if all([x in row.full_name for x in split1]) or all([x in row.fbref_name for x in split2]):
        return True
    else:
        return False

In [ ]:
merge_df['keep'] = merge_df.apply(filter_name, axis = 1)

In [ ]:
merge_df = merge_df[merge_df['keep']]

In [ ]:
merge_df['id'].duplicates()

NameError: name 'merge_df' is not defined

In [ ]:
# merge_df.to_csv('2022_merge.csv')

In [10]:
test = pd.read_csv('2022_merge.csv')

In [11]:
test

,Unnamed: 0,url_id,tag,fbref_name,key,first_name,second_name,id,full_name,keep
0,489,1f44ac21,Erling-Haaland,Erling Braut Haaland,0,Erling,Haaland,318,Erling Haaland,True
1,1252,e46012d4,Kevin-De-Bruyne,Kevin De Bruyne,0,Kevin,De Bruyne,301,Kevin De Bruyne,True
2,2755,5bc43860,Brenden-Aaronson,Brenden Russell Aaronson,0,Brenden,Aaronson,246,Brenden Aaronson,True
3,4531,f2bf1b0f,Che-Adams,Che Zach Everton Fred Adams,0,Che,Adams,411,Che Adams,True
4,5091,2b09d998,Tyler-Adams,Tyler Shaan Adams,0,Tyler,Adams,506,Tyler Adams,True
...,...,...,...,...,...,...,...,...,...,...
487,426569,384d58d9,Denis-Zakaria,Denis Zakaria,0,Denis,Zakaria,616,Denis Zakaria,True
488,427219,e8271ad9,Jordan-Zemura,Jordan Zemura,0,Jordan,Zemura,76,Jordan Zemura,True
489,428883,6622454d,Hakim-Ziyech,'Hakim Ziyech',0,Hakim,Ziyech,135,Hakim Ziyech,True
490,430172,ce4246f5,Kurt-Zouma,Kurt Happy Zouma,0,Kurt,Zouma,461,Kurt Zouma,True
